# CWB: Data Wrangling

**The Data Science Method**  


0.   Problem Identification 

1.   **Data Wrangling** 
  * Data Collection
      - Locating the data
      - Data loading
      - Data joining
  * Data Definition
      - Column names
      - Data types (numeric, categorical, timestamp, etc.)
      - Description of the columns
      - Count or percent per unique values or codes (including NA)
      - The range of values or codes  
  * Data Cleaning
      - NA or missing data
      - Duplicates
 
2.   Exploratory Data Analysis 

3.   Pre-processing and Training Data Development

4.   Modeling 

5.   Documentation

## Load Required Packages

In [1]:
# load python packages from environment
import os
import numpy as np
import pandas as pd

## Data Collection
Load the data into a pandas dataframe for ease of use.<br><br>
Raw data downloaded from the [World Air Quality Index Project](https://aqicn.org/city/hongkong/causeway-bay/) on February 15, 2020.

In [2]:
# Get current working directory
path = os.getcwd()

# Get parent directory
parent_path = os.path.dirname(path)
print(parent_path)

# Create data/raw path
rawdata_path = os.path.join(parent_path, 'data', 'raw')
print(rawdata_path)

/Users/tiffanyflor/Dropbox/MyProjects/Causeway Bay Pollution
/Users/tiffanyflor/Dropbox/MyProjects/Causeway Bay Pollution/data/raw


In [3]:
df = pd.read_csv(rawdata_path + '/causeway-bay-air-quality.csv')
df.head()

,date,pm25,pm10,o3,no2,so2,co
0,2021/2/2,64,45,31,41,3,6
1,2021/2/3,87,52,31,33,2,5
2,2021/2/4,85,52,32,33,2,4
3,2021/2/5,81,42,28,33,2,5
4,2021/2/6,71,46,5,53,4,6


In [4]:
df.columns = ['date','pm25','pm10','o3','no2','so2','co']

In [5]:
# According to the WAQIP website, the O3 is incorrect.
# NO2, SO2, and CO are missing a month and a half worth of data.
# Since we are only predicting pm25, we'll drop the incorrect and missing data columns.
df = df[['date','pm25','pm10']]
df.tail()

,date,pm25,pm10
2589,2015/1/1,,65
2590,2014/1/1,,79
2591,2018/7/23,,
2592,2018/7/24,,
2593,2018/7/25,,


## Data Definition
Examine columns, datatypes, column description, unique values, range of values. 

In [6]:
df.columns

Index(['date', 'pm25', 'pm10'], dtype='object')

In [7]:
df.dtypes

date    object
pm25    object
pm10    object
dtype: object

In [8]:
df.shape

(2594, 3)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2594 entries, 0 to 2593
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    2594 non-null   object
 1   pm25    2594 non-null   object
 2   pm10    2594 non-null   object
dtypes: object(3)
memory usage: 60.9+ KB


In [10]:
df.describe().T

,count,unique,top,freq
date,2594,2594,2016/5/15,1
pm25,2594,146,77,55
pm10,2594,83,33,83


In [11]:
df['pm25'].unique()

array([' 64', ' 87', ' 85', ' 81', ' 71', ' 93', ' 95', ' 97', ' 91',
       ' 48', ' 49', ' 67', ' 70', ' 73', ' 90', ' 100', ' 120', ' 96',
       ' 126', ' 92', ' 65', ' 77', ' 74', ' 89', ' 139', ' 124', ' 154',
       ' 140', ' 102', ' 98', ' 122', ' 118', ' 119', ' 75', ' 76', ' 82',
       ' 83', ' 52', ' 84', ' 72', ' 63', ' 86', ' 105', ' 79', ' 103',
       ' 66', ' 106', ' 111', ' 107', ' 109', ' 69', ' 62', ' 78', ' 80',
       ' 108', ' 138', ' 123', ' 88', ' 128', ' 101', ' 125', ' 110',
       ' 104', ' 35', ' 42', ' 54', ' 39', ' 36', ' 46', ' 53', ' 59',
       ' 45', ' 37', ' 33', ' 40', ' 47', ' 56', ' 50', ' 38', ' 43',
       ' 44', ' 60', ' 58', ' 57', ' 142', ' 51', ' 61', ' 155', ' 55',
       ' 41', ' 34', ' 99', ' 68', ' 94', ' 115', ' 113', ' 112', ' 30',
       ' 146', ' 23', ' 132', ' 114', ' 163', ' 162', ' 169', ' 151',
       ' 127', ' 136', ' 117', ' 130', ' 148', ' 121', ' 135', ' 131',
       ' 175', ' 116', ' 143', ' 129', ' 133', ' 137', ' 141', ' 1

## Data Cleaning
Review missing values, remove duplicates

In [12]:
# ' ' is used as the null value
df.isnull().sum()

date    0
pm25    0
pm10    0
dtype: int64

In [13]:
# replace ' ' with null
df = df.replace(' ', np.nan)
df.tail()

,date,pm25,pm10
2589,2015/1/1,NaN,65
2590,2014/1/1,NaN,79
2591,2018/7/23,NaN,NaN
2592,2018/7/24,NaN,NaN
2593,2018/7/25,NaN,NaN


### Order the timestamps correctly

In [14]:
df['date'] = pd.to_datetime(df['date'])
df.dtypes

date    datetime64[ns]
pm25            object
pm10            object
dtype: object

In [15]:
# Sort by date
df = df.sort_values(by='date',ascending=True).reset_index(drop=True)
df.head(3)

,date,pm25,pm10
0,2014-01-01,NaN,79
1,2014-01-02,166,66
2,2014-01-03,153,82


In [16]:
df.tail(3)

,date,pm25,pm10
2591,2021-02-14,70,31
2592,2021-02-15,73,37
2593,2021-02-16,90,NaN


### Create pm25 dataframe & interpolate missing values

In [17]:
pm25 = df[['date','pm25']].dropna().set_index('date')
pm25.head()

,pm25
date,
2014-01-02,166
2014-01-03,153
2014-01-04,158
2014-01-05,179
2014-01-06,175


In [18]:
# Interpolate missing values
pm25['pm25'] = pd.to_numeric(pm25['pm25'])
pm25_interpol = pm25.resample('D').mean()
pm25_interpol['pm25'] = pm25_interpol['pm25'].interpolate()

In [19]:
pm25_interpol.shape

(2603, 1)

### Create pm10 dataframe & interpolate missing values

In [20]:
pm10 = df[['date','pm10']].dropna().set_index('date')
pm10.head()

,pm10
date,
2014-01-01,79
2014-01-02,66
2014-01-03,82
2014-01-04,92
2014-01-05,90


In [21]:
# Interpolate missing values
pm10['pm10'] = pd.to_numeric(pm10['pm10'])
pm10_interpol = pm10.resample('D').mean()
pm10_interpol['pm10'] = pm10_interpol['pm10'].interpolate()

In [22]:
pm10_interpol.shape

(2603, 1)

### Concatenate Dataframes

In [23]:
df_final = pd.concat([pm25_interpol,pm10_interpol], axis=1)
df_final.head()

,pm25,pm10
date,,
2014-01-01,NaN,79.0
2014-01-02,166.0,66.0
2014-01-03,153.0,82.0
2014-01-04,158.0,92.0
2014-01-05,179.0,90.0


In [24]:
df_final.isnull().sum()

pm25    1
pm10    1
dtype: int64

In [25]:
# pm25's missing value is the first date
# we'll bfill
df_final['pm25'] = df_final['pm25'].bfill()
df_final.head(2)

,pm25,pm10
date,,
2014-01-01,166.0,79.0
2014-01-02,166.0,66.0


In [26]:
# pm10's missing value is the last date
df_final[df_final['pm10'].isnull()]

# forward fill
df_final['pm10'] = df_final['pm10'].ffill()
df_final.tail(2)

,pm25,pm10
date,,
2021-02-15,73.0,37.0
2021-02-16,90.0,37.0


### Check for duplicates

In [27]:
df_final.reset_index().duplicated().sum()

0

## Save csv to data/interim

In [28]:
df_final.head()

,pm25,pm10
date,,
2014-01-01,166.0,79.0
2014-01-02,166.0,66.0
2014-01-03,153.0,82.0
2014-01-04,158.0,92.0
2014-01-05,179.0,90.0


In [29]:
df_final.to_csv(parent_path + '/data/interim' + '/1.0_cwb_pm25_pm10_clean1.csv')